# Modeling Traffic Congestion using Weather

Accurate anticipation of traffic congestion is an important problem and can potentially yield immense benefits towards improving travel conditions for commuters as well as helping various industries that rely on ground transportation. Bad weather such as rain and snow is one of the biggest factors causing traffic congestion. In this notebook, we aim to investigate this problem and provide an overview of the steps that might be required to tackle it in an end-to-end manner, from data exploration to data transformation and using machine learning to obtain models that can be used to predict the real-time traffic congestion using historical traffic congestion and weather conditions.
In this notebook, we will use traffic and weather data from the city of Chicago. This data is available on the IBM Cloud Object Storage (COS). We will use IBM Cloud SQL Query to explore and transform this traffic and weather data to obtain useful data features that can then be used to train a simple machine learning model.

In most typical data science and machine learning pipelines, a number of data preparation steps involved before training predictive models. Particularly, one may want to explore the data to determine which variables might be useful, transform the data, and fuse and join data from multiple different sources. These are the high level steps that we will be following in this notebook as well. We will first explore, transform and join data leveraging various capabilities available through Cloud SQL Query, to obtain useful data features, which we will then use to build predictive models using scikit-learn.

## Table of contents
1. [Setup and Configuration](#setup)<br>
2. [Data Exploration using SQL Query](#exploration)<br>
3. [Step by step ETL - Spatiotemporal Joins using SQL Query](#etl)<br>
4. [Compond ETL](#compoundetl)<br>
5. [Machine Learning Model Training](#train)<br>
6. [Traffic Congestion Prediction](#prediction)<br>
7. [Predicted Traffic Congestion Visualization](#viz)<br>

### <a name="setup"></a> 1. Setup and Configuration

Let us first install all the dependencies that we will need and configure IBM Cloud SQL Query

In [32]:
%%capture
!conda install -y pyarrow
!pip install --user ibmcloudsql
!pip install folium
import ibmcloudsql
import getpass
from sklearn import tree
import folium
targeturl = ''

In [36]:
apikey=getpass.getpass('Enter IBM Cloud API Key (leave empty to use previous one): ') or apikey
instnacecrn=input('Enter SQL Query Instance CRN (leave empty to use previous one): ') or instnacecrn
if targeturl == '':
    targeturl=input('Enter target URL for SQL results: ')
else:
    targeturl=input('Enter target URL for SQL results (leave empty to use ' + targeturl + '): ') or targeturl
sqlClient = ibmcloudsql.SQLQuery(apikey, instnacecrn, client_info='Traffic Weather')
sqlClient.logon()

Enter IBM Cloud API Key (leave empty to use previous one): ········
Enter SQL Query Instance CRN (leave empty to use previous one): crn:v1:bluemix:public:sql-query:us-south:a/3ac874e10c8ca426580dc63d4462cbdd:a78efba7-0015-4262-98bc-397376ded16b::
Enter target URL for SQL results (leave empty to use cos://us-geo/sqltest/result): 


### <a name="exploration"></a> 2. Data Exploration using SQL Query

The data that we will use in this notebook is stored in *4 different objects in COS*. 

- **cos://us-geo/sql/segment_locations.parquet** contains the **metadata for each of the road segments**

- **cos://us-geo/sql/traffic.parquet** contains the **historical traffic congestion data for each of the road segments**

- **cos://us-geo/sql/weather_sites.parquet** contains the **metadata for each of the weather sites**

- **cos://us-geo/sql/weather.parquet** contains the **historical weather data for each of the weather sites**

Let us take a quick look at the data.

**What does the traffic data look like?**

In [37]:
sql = "SELECT SegmentID, ST_AsText(Segment) as RoadSegment FROM cos://us-east/mytestbucket123/segment_locations.parquet STORED AS PARQUET INTO {}".format(targeturl)
traffic_meta_explore = sqlClient.run_sql(sql)
if isinstance(traffic_meta_explore, str):
    print(traffic_meta_explore)
traffic_meta_explore.head(10)

,SegmentID,RoadSegment
0,1088,"LINESTRING (-87.6413654096 41.6634679215, -87...."
1,148,"LINESTRING (-87.6662077421 41.8523321221, -87...."
2,463,"LINESTRING (-87.6727255878 41.7211640089, -87...."
3,496,"LINESTRING (-87.7526573727 41.7970886104, -87...."
4,1238,"LINESTRING (-87.644021 41.743359, -87.643756 4..."
5,471,"LINESTRING (-87.594731404 41.7222769494, -87.6..."
6,1084,"LINESTRING (-87.7191682775 41.6622990645, -87...."
7,897,"LINESTRING (-87.6537610863 41.9855554936, -87...."
8,858,"LINESTRING (-87.7782350055 41.9733282922, -87...."
9,1025,"LINESTRING (-87.6367064128 41.8819140751, -87...."


In [38]:
sql = "SELECT COUNT(DISTINCT SegmentID) FROM cos://us-east/mytestbucket123/segment_locations.parquet STORED AS PARQUET INTO {}".format(targeturl)
segment_id_count = sqlClient.run_sql(sql)
if isinstance(segment_id_count, str):
    print(segment_id_count)
segment_id_count.head()

,count(DISTINCT SegmentID)
0,1257


In [39]:
sql = "SELECT * FROM cos://us-geo/sql/traffic.parquet STORED AS PARQUET LIMIT 10 INTO {}".format(targeturl)
traffic_explore = sqlClient.run_sql(sql)
if isinstance(traffic_explore, str):
    print(traffic_explore)
traffic_explore.head(10)

,TIME,SEGMENTID,SPEED
0,04/09/2018 11:10:30 PM,1090,-1
1,04/09/2018 08:40:14 AM,57,20
2,04/09/2018 10:20:49 PM,1232,-1
3,04/09/2018 06:21:05 PM,960,11
4,04/10/2018 03:10:32 PM,300,21
5,04/08/2018 08:50:20 PM,187,21
6,04/10/2018 10:40:06 AM,4,23
7,04/08/2018 06:01:20 PM,967,-1
8,04/08/2018 02:40:12 PM,294,19
9,04/10/2018 08:31:05 PM,893,42


**What does the weather data look like?**

In [40]:
sql = "SELECT * FROM cos://us-geo/sql/weather_sites.parquet STORED AS PARQUET INTO {}".format(targeturl)
weather_meta_explore = sqlClient.run_sql(sql)
if isinstance(weather_meta_explore, str):
    print(weather_meta_explore)
weather_meta_explore.head(10)

,SiteId,Latitude,Longitude
0,2265659164,42.43317,-88.50244
1,2260359280,42.11351,-87.24051
2,2245275119,41.21886,-88.31384
3,2252346520,41.63551,-87.10202
4,2255084035,41.79811,-87.72607
5,2247889779,41.37234,-87.21756
6,2255941743,41.84923,-87.35632
7,2250636828,41.53433,-87.83850
8,2257748129,41.95708,-88.35229
9,2264752903,42.37830,-87.12344


In [41]:
sql = "SELECT COUNT(DISTINCT SiteID) FROM cos://us-geo/sql/weather_sites.parquet STORED AS PARQUET INTO {}".format(targeturl)
weather_site_count = sqlClient.run_sql(sql)
if isinstance(weather_site_count, str):
    print(weather_site_count)
weather_site_count.head()

,count(DISTINCT SiteID)
0,22


In [42]:
sql = "SELECT * FROM cos://us-geo/sql/weather.parquet STORED AS PARQUET INTO {}".format(targeturl)
weather_explore = sqlClient.run_sql(sql)
if isinstance(weather_explore, str):
    print(weather_explore)
weather_explore.head(10)

,SiteId,DateHrGmt,SurfaceTemperatureFahrenheit,SurfaceDewpointTemperatureFahrenheit,SurfaceWetBulbTemperatureFahrenheit,RelativeHumidityPercent,SurfaceAirPressureMillibars,CloudCoveragePercent,WindChillTemperatureFahrenheit,ApparentTemperatureFahrenheit,...,WindDirectionDegrees,PrecipitationPreviousHourInches,DownwardSolarRadiationWsqm,DiffuseHorizontalRadiationWsqm,DirectNormalIrradianceWsqm,MslPressureMillibars,HeatIndexFahrenheit,SnowfallInches,SurfaceWindGustsMph,ReferenceEvapotranspirationInchesPerHour
0,2260359280,04/09/2018 18:00:00,32.3,27.7,30.5,83,1002.0,17,23.4,23.4,...,14,0.000,909,92,994,1023.0,32.3,0.0,18.3,0.000315
1,2258638047,04/12/2018 15:00:00,61.3,54.9,57.4,80,977.0,72,61.3,61.3,...,250,0.000,593,80,902,1002.0,61.3,0.0,32.9,0.000512
2,2263045486,04/06/2018 02:00:00,33.0,28.6,31.3,84,991.0,100,24.3,24.3,...,168,0.010,0,0,0,1017.0,33.0,0.0,20.6,0.000000
3,2258638047,04/01/2018 04:00:00,31.4,26.5,29.6,82,995.0,100,21.5,21.5,...,319,0.000,0,0,0,1022.0,31.4,0.0,41.4,0.000000
4,2250636828,04/09/2018 01:00:00,32.3,19.8,28.2,60,994.0,100,25.8,25.8,...,73,0.001,0,0,0,1019.0,32.3,0.0,12.1,0.000039
5,2255084035,04/03/2018 03:00:00,36.8,31.3,34.4,80,988.0,100,29.7,29.7,...,116,0.000,0,0,0,1012.0,36.8,0.0,15.7,0.000039
6,2263909103,04/08/2018 17:00:00,29.9,20.0,26.7,66,999.0,1,22.4,22.4,...,77,0.000,866,89,1004,1021.0,29.9,0.0,12.6,0.000354
7,2245275119,04/06/2018 19:00:00,35.2,24.5,31.3,65,993.0,98,24.4,24.4,...,288,0.000,529,411,145,1016.0,35.2,0.0,44.7,0.000276
8,2246165668,04/03/2018 20:00:00,40.5,39.9,40.2,98,975.0,100,33.6,33.6,...,27,0.120,23,23,0,998.0,40.5,0.0,40.6,0.000157
9,2259508022,04/07/2018 10:00:00,24.0,16.9,22.0,75,999.0,18,11.4,11.4,...,331,0.000,0,0,0,1022.0,24.0,0.0,25.5,0.000000


In summary, we have **historical traffic congestion data for total of 1257 road segments** and **weather data for 22 weather sites**, stored in the following objects:

**(1) Traffic Metadata** - This object contains metadata related to road segments, i.e, the road segment identifiers and the line segment geometry for the road segments.

**(2) Traffic Data** - This object contains traffic data for road segments from the city of Chicago. Specifically, it consists of the road segment identifiers, timestamps, and speeds, with a sampling rate of 10 minutes. Congestion on a road segment at a given instance in time is defined as the (speed / reference_speed). We will assume the reference speed to be 30 mph. A congestion value less than 1 means that there is no congestion, whereas a value greater than 1 implies a congested road.

**(3) Weather Metadata** - This object contains metadata related to weather stations, i.e, the weather station identifiers and their location (represented by the latitude and longitude) in the city of Chicago.

**(4) Weather Data** - This object contains weather data for weather sites. Specifically, it consists of weather station identifiers, timestamps, and  values for various weather attributes, including surface temperature, precipitation, snowfall, wind speed, etc.


### <a name="etl"></a> 3. ETL - Spatiotemporal Joins using SQL Query

Now that we know that the traffic and weather data resides in multiple different buckets, we need to join this data together to obtain data features for our predictive machine learning model. We can accomplish this using the various spatial and temporal join functions that SQL Query provides. We can do this ETL in a single compound query or break it down into a multiple steps. In this section, we use the step by step approach to better explain the process. [Section 4](#compoundetl) has the same steps in a single compound query. So, one could also skip this section entirely and directly run the cell in [section 4](#compoundetl). If you run cells in this section, there is no need to run the cells in the next section since they both have the same effect.

**(1) Join Traffic and Weather Metadata for Road Segments in the city of Chicago and nearby suburbs**: We first sub-select segment locations in Chicago and its neighboring suburbs through a `ST_Contains` spatial operation. We then perform a spatial distance based cross join between the sub-selected segment locations and the weather site locations using the `ST_Distance` spatial predicate and pick the closest distance pairs to obtain corresponding weather site identifiers for every road segment identifier.

In [43]:
metadata_join_sql = """
SELECT SegmentID,
	SiteId FROM
		(SELECT *, dense_rank() OVER (PARTITION BY SegmentID
                                      ORDER BY dist ASC) AS rank FROM
					(SELECT SegmentID, SiteId, ST_Distance(SEGMENT, ST_Point(Longitude, Latitude)) AS dist
						FROM cos://us-geo/sql/weather_sites.parquet STORED AS PARQUET
						CROSS JOIN
								(SELECT SegmentID, SEGMENT
									FROM cos://us-east/mytestbucket123/segment_locations.parquet STORED AS PARQUET
									WHERE ST_Contains(ST_WKTToSQL('POLYGON((-87.66182361726857 42.013649516597454,-87.78404651765919 42.00956804496257,-87.78541980867482 41.90335786993515,-87.71126209383107 41.74268615114427,-87.61513172273732 41.71296382783499,-87.54784046297169 41.746784702779486,-87.61650501375294 41.86246067685847,-87.66182361726857 42.013649516597454))'), SEGMENT))))
WHERE rank <= 1 INTO {}
""".format(targeturl)
metadata_join_jobId = sqlClient.submit_sql(metadata_join_sql)
job_status = sqlClient.wait_for_job(metadata_join_jobId)
metadata_join_location = sqlClient.get_job(metadata_join_jobId)['resultset_location']

**(2) Create Traffic Congestion Timeseries for each Road Segment**: We create per- road segment traffic congestion timeseries using the `TIME_SERIES` function. The dataset consists of -1 speeds when no data is available at a given time. We will filter out these values.

In [44]:
traffic_ts_sql = """
SELECT SegmentID,
	TIME_SERIES(TS_TIMESTAMP(to_timestamp(TIME, "MM/dd/yyyy hh:mm:ss aaa")), speed/30) AS ts
FROM cos://us-geo/sql/traffic.parquet STORED AS PARQUET
WHERE speed >= 0
GROUP BY SegmentID INTO {} STORED AS PARQUET
""".format(targeturl)
traffic_ts_jobId = sqlClient.submit_sql(traffic_ts_sql)
job_status = sqlClient.wait_for_job(traffic_ts_jobId)
traffic_ts_location = sqlClient.get_job(traffic_ts_jobId)['resultset_location']

**(3) Create Precipitation Timeseries for each Weather Site**: We create per- weather site timeseries for precipitation values using the `TIME_SERIES` function.

In [45]:
precipitation_ts_sql = """
SELECT SiteId,
	TIME_SERIES(TS_TIMESTAMP(to_timestamp(DateHrGmt, "MM/dd/yyyy HH:mm:ss")), PrecipitationPreviousHourInches) AS ts
FROM cos://us-geo/sql/weather.parquet STORED AS PARQUET
GROUP BY SiteId INTO {} STORED AS PARQUET
""".format(targeturl)
precipitation_ts_jobId = sqlClient.submit_sql(precipitation_ts_sql)
job_status = sqlClient.wait_for_job(precipitation_ts_jobId)
precipitation_ts_location = sqlClient.get_job(precipitation_ts_jobId)['resultset_location']

**(4) Create Snow Timeseries for each Weather Site**: We create per- weather site timeseries for snow values using the `TIME_SERIES` function.

In [46]:
snow_ts_sql = """
SELECT SiteId, 
        TIME_SERIES(TS_TIMESTAMP(to_timestamp(DateHrGmt, "MM/dd/yyyy HH:mm:ss")), SnowfallInches) AS ts
FROM cos://us-geo/sql/weather.parquet STORED AS PARQUET
GROUP BY SiteId INTO {} STORED AS PARQUET
""".format(targeturl)
snow_ts_jobId = sqlClient.submit_sql(snow_ts_sql)
job_status = sqlClient.wait_for_job(snow_ts_jobId)
snow_ts_location = sqlClient.get_job(snow_ts_jobId)['resultset_location']

**(5) Join the Traffic Congestion Timeseries with the table obtained in step (1)**: In this step, we will join the traffic congestion timeseries obtained in step (2) with the metadata obtained in step (1), so that we have closest weather site identifier for every road segment identifier.

In [47]:
metadata_traffic_ts_join_sql = """
SELECT traffic_ts.SegmentID, 
        traffic_ts.ts, 
        metadata.SiteId 
FROM {} AS metadata, 
        {} STORED AS PARQUET AS traffic_ts
WHERE traffic_ts.SegmentID = metadata.SegmentID INTO {} STORED AS PARQUET
""".format(metadata_join_location, traffic_ts_location, targeturl)
metadata_traffic_ts_join_jobId = sqlClient.submit_sql(metadata_traffic_ts_join_sql)
job_status = sqlClient.wait_for_job(metadata_traffic_ts_join_jobId)
metadata_traffic_ts_join_location = sqlClient.get_job(metadata_traffic_ts_join_jobId)['resultset_location']

**(6) Join the Traffic Congestion Timeseries and the Precipitation Weather timeseries**: We join the traffic congestion timeseries with metadata  obtained in step (5) with the precipitation timeseries obtained in step (3) using the `TS_LEFT_JOIN` timeseries function. We use a left join here because the traffic congestion timeseries are sampled at a higher rate than the weather site timeseries. To interpolate any missing or null values in the precipitation timeseries, we use the `TS_INTERPOLATOR_PREV` interpolation function.

In [48]:
traffic_ts_join_precipitation_ts_sql = """
SELECT SegmentID, 
        precipitation_ts.SiteId, 
        TS_LEFT_JOIN(id_joined_traffic_ts.ts, precipitation_ts.ts, TS_INTERPOLATOR_PREV(0.0)) as ts1_ts2
FROM {} STORED AS PARQUET AS id_joined_traffic_ts, 
        {} STORED AS PARQUET AS precipitation_ts
WHERE id_joined_traffic_ts.SiteId = precipitation_ts.SiteId INTO {} STORED AS PARQUET
""".format(metadata_traffic_ts_join_location, precipitation_ts_location, targeturl)
traffic_ts_join_precipitation_ts_jobId = sqlClient.submit_sql(traffic_ts_join_precipitation_ts_sql)
job_status = sqlClient.wait_for_job(traffic_ts_join_precipitation_ts_jobId)
traffic_ts_join_precipitation_ts_location = sqlClient.get_job(traffic_ts_join_precipitation_ts_jobId)['resultset_location']

**(7) Join the Traffic Congestion Timeseries and the Snow Weather timeseries**: We join the traffic congestion timeseries with metadata obtained in step (5) with the snow timeseries obtained in step (4) using the `TS_LEFT_JOIN` timeseries function. To interpolate any missing or null values in the snow timeseries, we use the `TS_INTERPOLATOR_PREV` interpolation function.

In [49]:
traffic_ts_join_snow_ts_sql = """
SELECT SegmentID, 
        snow_ts.SiteId, 
        TS_LEFT_JOIN(id_joined_traffic_ts.ts, snow_ts.ts, TS_INTERPOLATOR_PREV(0.0)) as ts1_ts2
FROM {} STORED AS PARQUET AS id_joined_traffic_ts,
        {} STORED AS PARQUET AS snow_ts
WHERE id_joined_traffic_ts.SiteId = snow_ts.SiteId INTO {} STORED AS PARQUET
""".format(metadata_traffic_ts_join_location, snow_ts_location, targeturl)
traffic_ts_join_snow_ts_jobId = sqlClient.submit_sql(traffic_ts_join_snow_ts_sql)
job_status = sqlClient.wait_for_job(traffic_ts_join_snow_ts_jobId)
traffic_ts_join_snow_ts_location = sqlClient.get_job(traffic_ts_join_snow_ts_jobId)['resultset_location']

**(8) Perform Temporal Join on on the previously aligned timeseries**: We join the three timeseries, traffic congestion, precipitation and snow together using the `TS_VECTOR3` timeseries function. The `TS_TAKE_INDEX` function gets the appropriate timeseries from the previously joined timeseries.

In [50]:
join_all_ts_sql = """
SELECT traffic_precipitation_joined_ts.SegmentID, 
        TS_VECTOR3(TS_INDEX(traffic_precipitation_joined_ts.ts1_ts2, 0), 
                    TS_INDEX(traffic_precipitation_joined_ts.ts1_ts2, 1), 
                    TS_INDEX(traffic_snow_joined_ts.ts1_ts2, 1))
        AS joined_ts 
FROM {} STORED AS PARQUET AS traffic_precipitation_joined_ts, 
        {} STORED AS PARQUET AS traffic_snow_joined_ts 
WHERE traffic_precipitation_joined_ts.SegmentID = traffic_snow_joined_ts.SegmentID INTO {} STORED AS PARQUET
""".format(traffic_ts_join_precipitation_ts_location, traffic_ts_join_snow_ts_location, targeturl)
join_all_ts_jobId = sqlClient.submit_sql(join_all_ts_sql)
job_status = sqlClient.wait_for_job(join_all_ts_jobId)
join_all_ts_location = sqlClient.get_job(join_all_ts_jobId)['resultset_location']

**(9) Explode Temporally Joined Series in step (8) and select the road segment identifier, congestion and weather features for each timestamp ordered by the (identifier, timestamp) to get the final set of features to train the machine learning model**: Finally, we explode the previously joined timeseries to get the joined traffic congestion and weather features.

In [51]:
features_sql = """
WITH joined_exploded AS
		(SELECT SegmentID,
				TS_DA_EXPLODE(joined_ts) AS (TIMESTAMP,value)
			FROM {} STORED AS PARQUET AS traffic_weather_joined_ts)
SELECT SegmentID,
	TIMESTAMP,
	value[0] AS congestion,
	value[1] AS precipitation,
	value[2] AS snow
FROM joined_exploded
ORDER BY SegmentID,
	TIMESTAMP
INTO {}
""".format(join_all_ts_location, targeturl + "_train_features")
features = sqlClient.run_sql(features_sql)
if isinstance(features, str):
    print(features)

In [52]:
features.head(10)

,SegmentID,TIMESTAMP,congestion,precipitation,snow
0,1,1522540866000,0.833333,0.0,0.0
1,1,1522542038000,0.833333,0.0,0.0
2,1,1522543205000,0.966667,0.0,0.0
3,1,1522546806000,0.833333,0.0,0.0
4,1,1522547416000,0.966667,0.0,0.0
5,1,1522548619000,0.833333,0.0,0.0
6,1,1522549238000,0.900000,0.0,0.0
7,1,1522549849000,0.833333,0.0,0.0
8,1,1522559416000,1.033333,0.0,0.0
9,1,1522561202000,1.066667,0.0,0.0


### <a name="compoundetl"></a> 4. Compound ETL
For efficiency reasons, instead of running step by step ETL statements and storing and retrieving intermediate results from COS, we can directly run our entire ETL pipeline in a single SQL query. We will look at how to do that in this section. You can skip running the cell in this section if you have already run the cells in the previous section.

In [185]:
compound_etl_sql = """
-- ### (1) JOIN TRAFFIC AND WEATHER METADATA IDENTIFIERS: WE FIRST SUB-SELECT SEGMENT LOCATIONS IN DOWNTOWN CHICAGO THROUGH A ST_CONTAINS SPATIAL 
    -- OPERATION. WE THEN PERFORM A SPATIAL DISTANCE BASED CROSS JOIN BETWEEN THE SUB-SELECTED SEGMENT LOCATIONS AND THE WEATHER SITE LOCATIONS AND PICK 
    -- THE CLOSEST DISTANCE PAIRS TO OBTAIN CORRESPONDING WEATHER SITE IDS FOR EVERY SEGMENT ID. ###
WITH
    metadata AS (SELECT SegmentID, SiteId
                    FROM
                (
                SELECT *, dense_rank() over (partition by SegmentID order by dist asc) as rank
                    FROM
                (
                SELECT SegmentID, SiteId, ST_Distance(Segment, ST_Point(Longitude, Latitude)) as dist 
                    FROM cos://us-geo/sql/weather_sites.parquet STORED AS PARQUET 
                    CROSS JOIN (SELECT SegmentID, Segment FROM cos://us-east/mytestbucket123/segment_locations.parquet STORED AS PARQUET
                    WHERE ST_Contains(ST_WKTToSQL('POLYGON((-87.66182361726857 42.013649516597454,-87.78404651765919 42.00956804496257,-87.78541980867482 41.90335786993515,-87.71126209383107 41.74268615114427,-87.61513172273732 41.71296382783499,-87.54784046297169 41.746784702779486,-87.61650501375294 41.86246067685847,-87.66182361726857 42.013649516597454))'), Segment))
                )
                )
                WHERE rank <= 1
                ),
    -- ### (2) CREATE TRAFFIC TIMESERIES: WE CREATE PER- ROAD SEGMENT TRAFFIC CONGESTION TIMESERIES USING THE TIME_SERIES FUNCTION. ###
    traffic_ts AS (SELECT SegmentID, TIME_SERIES(TS_TIMESTAMP(to_timestamp(TIME, "MM/dd/yyyy hh:mm:ss aaa")), speed/30) AS ts
                          FROM cos://us-geo/sql/traffic.parquet STORED AS PARQUET
                          WHERE speed >= 0
                          GROUP BY SegmentID),
    -- ### (3) CREATE A PER- WEATHER SITE TIMESERIES FOR PRECIPITATION VALUES USING THE TIME_SERIES FUNCTION. ###
    precipitation_ts AS (SELECT SiteId, TIME_SERIES(TS_TIMESTAMP(to_timestamp(DateHrGmt, "MM/dd/yyyy HH:mm:ss")), PrecipitationPreviousHourInches) AS ts
                          FROM cos://us-geo/sql/weather.parquet STORED AS PARQUET
                          GROUP BY SiteId),
    -- ### (4) CREATE A PER- WEATHER SITE TIMESERIES FOR SNOW VALUES USING THE TIME_SERIES FUNCTION. ###
    snow_ts AS (SELECT SiteId, TIME_SERIES(TS_TIMESTAMP(to_timestamp(DateHrGmt, "MM/dd/yyyy HH:mm:ss")), SnowfallInches) AS ts
                          FROM cos://us-geo/sql/weather.parquet STORED AS PARQUET
                          GROUP BY SiteId),
    -- ### (5) JOIN THE TRAFFIC CONGESTION TIMESERIES WITH THE METADATA OBTAINED IN STEP (1), SO THAT WE HAVE CLOSEST SITE ID FOR EVERY SEGMENT ID. ### 
    id_joined_traffic_ts AS (SELECT traffic_ts.SegmentID, traffic_ts.ts, metadata.SiteId FROM metadata, traffic_ts 
                             WHERE traffic_ts.SegmentID = metadata.SegmentID),
    -- ### (6) JOIN THE TRAFFIC CONGESTION TIMESERIES WITH THE PRECIPITATION TIMESERIES USING THE TS_LEFT_JOIN FUNCTION. ###
    traffic_precipitation_joined_ts AS (SELECT SegmentID, precipitation_ts.SiteId, 
                                                TS_LEFT_JOIN(id_joined_traffic_ts.ts, precipitation_ts.ts, TS_INTERPOLATOR_PREV(0.0)) as ts1_ts2
                                        FROM id_joined_traffic_ts, precipitation_ts
                                        WHERE id_joined_traffic_ts.SiteId = precipitation_ts.SiteId),
    -- ### (7) JOIN THE TRAFFIC CONGESTION TIMESERIES WITH THE SNOW TIMESERIES USING THE TS_LEFT_JOIN FUNCTION. ###
    traffic_snow_joined_ts AS (SELECT SegmentID, snow_ts.SiteId, 
                                    TS_LEFT_JOIN(id_joined_traffic_ts.ts, snow_ts.ts, TS_INTERPOLATOR_PREV(0.0)) as ts1_ts2
                                FROM id_joined_traffic_ts, snow_ts
                                WHERE id_joined_traffic_ts.SiteId = snow_ts.SiteId),
    -- ### (8) JOIN THE TRAFFIC CONGESTION, PRECIPITATION AND SNOW TIMESERIES TOGETHER. ###
    traffic_weather_joined_ts AS (SELECT traffic_precipitation_joined_ts.SegmentID, 
                                    TS_VECTOR3(TS_INDEX(traffic_precipitation_joined_ts.ts1_ts2, 0), 
                                                TS_INDEX(traffic_precipitation_joined_ts.ts1_ts2, 1), 
                                                TS_INDEX(traffic_snow_joined_ts.ts1_ts2, 1)) AS joined_ts 
                                FROM traffic_precipitation_joined_ts, traffic_snow_joined_ts 
                                WHERE traffic_precipitation_joined_ts.SegmentID = traffic_snow_joined_ts.SegmentID),
    -- ### (9) EXPLODE THE PREVIOUSLY JOINED TIMESERIES TO GET THE JOINED CONGESTION AND WEATHER FEATURES. ###
    joined_exploded AS (SELECT SegmentID, TS_DA_EXPLODE(joined_ts) AS (timestamp, value)
                        FROM traffic_weather_joined_ts)
SELECT SegmentID, timestamp, value[0] AS congestion, value[1] AS precipitation, value[2] as snow FROM joined_exploded order by SegmentID, timestamp
INTO {}
""".format(targeturl + "_train_features")
features = sqlClient.run_sql(compound_etl_sql)
if isinstance(features, str):
    print(features)
features.head(10)

,SegmentID,timestamp,congestion,precipitation,snow
0,37,1522962067000,0.833333,0.001,0.0
1,37,1522962620000,0.666667,0.001,0.0
2,37,1522963237000,0.900000,0.001,0.0
3,37,1522963853000,0.633333,0.001,0.0
4,37,1522965019000,0.766667,0.001,0.0
5,37,1522965670000,0.600000,0.001,0.0
6,37,1522966220000,0.700000,0.001,0.0
7,37,1522966836000,0.666667,0.001,0.0
8,37,1522967450000,0.733333,0.001,0.0
9,37,1522968618000,0.633333,0.001,0.0


### <a name="train"></a> 5. Machine Learning Model Training

We now have all the required data features per-road segment, per-timestamp needed to train our predictive model, in the 'features' dataframe. Next, we will group these congestion and weather features by road segment identifiers, so that we can create per-road segment models. Specifically, we will develop regression tree models for each road segment using the scikit-learn package. The training involves regressing on the traffic congestion and weather condition values to predict the traffic congestion at the next timestep (since we have a sampling rate of 10 minutes, next (predicted) timestep will be 10 minutes ahead).

In [53]:
all_models = {}
grouped = features.groupby('SegmentID')
# Loop through all road segments one by one, since we need to create one model for each road segment
for name, group in grouped:
    x = []
    y = []
    # For each road segment, create data instances to train a regression tree, by looping through every 10 minute sample
    for i in range(0, group.shape[0] - 1):
        prev_congestion = group.iloc[i]['congestion']
        snow = group.iloc[i]['snow']
        precip = group.iloc[i]['precipitation']
        predict_congestion = group.iloc[i+1]['congestion']
        x.append([prev_congestion, snow, precip])
        y.append(predict_congestion)
    clf = tree.tree.DecisionTreeRegressor()
    if len(x) > 0:
        clf = clf.fit(x, y)
        all_models[name] = clf    

In the cell above, we created per-road segment predictive models. These are stored in the 'all_models' dictionary keyed by the road segment identifier, so that we can use them to predict traffic congestion later.

### <a name="prediction"></a> 6. Traffic Congestion Prediction

Now that we have trained machine learning models to predict traffic congestion on road segments, we can use them to predict congestion in real-time based on the traffic and weather conditions.
For simplicity, in this notebook, we will show a single prediction at the next time step, i.e, for each road segment, we will use the available traffic congestion and weather attributes from the last timestep in 'features' as inputs and predict congestion at the next timestep

In [54]:
# Add last timestep traffic congestion and weather values as input values to be used for prediction
to_predict = {}
for name, group in grouped:
    last_timestep = group.shape[0] - 1
    to_predict[name] = [[group.iloc[last_timestep]['congestion'], group.iloc[last_timestep]['snow'], group.iloc[last_timestep]['precipitation']]]
# Get the per road segment predictions using inputs in 'to_predict' and also store them in a dictionary
predicted = {}
for val in to_predict:
    if val in all_models:
        pred_congestion = all_models[val].predict(to_predict[val])
        predicted[val] = pred_congestion
        print('Predicted congestion for segment ' + str(val) + " = " + str(pred_congestion[0]))

Predicted congestion for segment 1 = 0.729166666667
Predicted congestion for segment 2 = 0.766666666667
Predicted congestion for segment 3 = 0.73567251462
Predicted congestion for segment 4 = 0.704301075269
Predicted congestion for segment 5 = 0.913402061856
Predicted congestion for segment 6 = 0.696396396396
Predicted congestion for segment 7 = 0.71875
Predicted congestion for segment 8 = 0.71746031746
Predicted congestion for segment 9 = 0.98115942029
Predicted congestion for segment 10 = 0.746846846847
Predicted congestion for segment 11 = 0.792063492063
Predicted congestion for segment 29 = 0.75
Predicted congestion for segment 30 = 0.849450549451
Predicted congestion for segment 31 = 0.789523809524
Predicted congestion for segment 32 = 0.72697495183
Predicted congestion for segment 33 = 0.782777777778
Predicted congestion for segment 34 = 0.878395061728
Predicted congestion for segment 35 = 0.896875
Predicted congestion for segment 36 = 1.02276422764
Predicted congestion for segme

### <a name="viz"></a> 7. Predicted Traffic Congestion Visualization

While we can see the congestion values for each of the road segments above, traffic congestion on roads is much better understood when visualized on a map. This is exactly what we will do next. But since the road segment geometries were not part of our data features, we need to obtain them from COS. For this, we can once again use a SQL query to get the geometries for road segments in Chicago, which is where our road segments were from.

In [55]:
sql = """
SELECT SegmentID, ST_Y(ST_StartPoint(Segment)) as start_lat,
                  ST_X(ST_StartPoint(Segment)) as start_lon,
                  ST_Y(ST_EndPoint(Segment)) as end_lat, 
                  ST_X(ST_EndPoint(Segment)) as end_lon FROM cos://us-east/mytestbucket123/segment_locations.parquet STORED AS PARQUET
                    WHERE ST_Contains(ST_WKTToSQL('POLYGON((-87.66182361726857 42.013649516597454,-87.78404651765919 42.00956804496257,-87.78541980867482 41.90335786993515,-87.71126209383107 41.74268615114427,-87.61513172273732 41.71296382783499,-87.54784046297169 41.746784702779486,-87.61650501375294 41.86246067685847,-87.66182361726857 42.013649516597454))'), Segment) INTO {}
""".format(targeturl)

In [56]:
segment_locations = sqlClient.run_sql(sql)
if isinstance(segment_locations, str):
    print(segment_locations)

'segment_locations' contains the road segment identifiers as well as the corresponding line segment geometries for those road segments. We also have the per road segment congestion predictions in the 'predicted' dictionary that we obtained in the last step. We will use these road segment geometries and predictions to visualize the results. 

For each major road segment in downtown chicago, **a traffic congestion value of or greater than 1.5 is denoted by a red color line segment**, a **traffic congestion value between 1.0 and 1.5 is denoted by an orange color line segment** and **a traffic congestion value of or less than 1.0 is denoted by a green color line segment**.

In [57]:
m = folium.Map([41.867169, -87.678114], zoom_start=11)
for index, row in segment_locations.iterrows():
    locations = [[row['start_lat'], row['start_lon']], [row['end_lat'], row['end_lon']]]
    if row['SegmentID'] in predicted:
        val = row['SegmentID']
        congestion = predicted[val]
        color = 'green'
        if congestion > 1.0 and congestion < 1.5:
            color = 'orange'
        elif congestion >= 1.5:
            color = 'red'
        folium.PolyLine(locations,weight=5,color=color).add_to(m)
m   